In [6]:
config = {'serverless': {'backend': 'ibm_cf'},
          'ibm_cf':  {'serverless': {'backend': 'ibm_cf'},
                      'endpoint': 'https://eu-gb.functions.cloud.ibm.com',
                      'namespace': 'cloudbutton@uvigo.es_dev',
                      'api_key': 'c8a9e3ec-51c9-413b-ac23-e10c3ccb71e1:k3GoGB7GRgyNFYI3ob97GAuKt8ORPYJ9eWjCfIygD0d2xeR9aowjaQlvgm7HhlPm'},
          'ibm_cos': {'endpoint': 'https://s3.eu-gb.cloud-object-storage.appdomain.cloud', #https://s3.eu.cloud-object-storage.appdomain.cloud
                      'private_endpoint': 'https://s3.private.eu-gb.cloud-object-storage.appdomain.cloud',
                      'api_key' : '0GRleyXkQsvdhokMlmI0Ve-p7WxAhnKCJz9F-M7bu9qe'},
          'lithops' : {'storage_bucket' : 'bucket-gerard-eu-gb'}}

In [99]:
import io
import rasterio
from rasterio.plot import reshape_as_image
import joblib
import numpy as np

In [86]:
raster_path = '/home/jovyan/work/Dataset_valencia/Dataset_processing/Tiling/valenciana_656000.0-4336000.0_comunidad_mask.tif'
print(raster_path)

/home/jovyan/work/Dataset_valencia/Dataset_processing/Tiling/valenciana_656000.0-4336000.0_comunidad_mask.tif


In [87]:
raster = rasterio.open(raster_path)

In [96]:
img = raster.read()
img
img.shape

(3, 1599, 1599)

In [97]:
reshaped_img = reshape_as_image(img)
reshaped_img.shape

(1599, 1599, 3)

# Stackering try

In [57]:
import warnings
import matplotlib.pyplot as plt
import numpy as np
import lithops
import pickle
import os
import rasterio
from rasterio.plot import reshape_as_image
from rasterio.plot import show
warnings.filterwarnings('ignore')

In [61]:
def map_stackering(b1, b2, b3, r1, ibm_cos): # pasar aqui un diccionario con el modelo y la imagen
   
    b1 = ibm_cos.get_object(Bucket='testing-lithops', Key='sentinel_stackering/SENTINEL2A_20200927-110006-031_L2A_T30TYK_C_V2-2_SRE_B3.tif')['Body']
    raster = rasterio.open(b1)
    img = raster.read()
    reshaped_img = reshape_as_image(img)
    
    i=2

    return reshaped_img

In [62]:
iterdata  = [('cos://testing-lithops/sentinel_stackering/SENTINEL2A_20200927-110006-031_L2A_T30TYK_C_V2-2_SRE_B3.tif',
             'cos://testing-lithops/sentinel_stackering/SENTINEL2A_20200927-110006-031_L2A_T30TYK_C_V2-2_SRE_B4.tif',
             'cos://testing-lithops/sentinel_stackering/SENTINEL2A_20200927-110006-031_L2A_T30TYK_C_V2-2_SRE_B8.tif',
             'cos://testing-lithops/sentinel_stackering/empty.tif'),
            ('cos://testing-lithops/sentinel_stackering/SENTINEL2A_20200927-110006-031_L2A_T30TYK_C_V2-2_SRE_B3.tif',
             'cos://testing-lithops/sentinel_stackering/SENTINEL2A_20200927-110006-031_L2A_T30TYK_C_V2-2_SRE_B4.tif',
             'cos://testing-lithops/sentinel_stackering/SENTINEL2A_20200927-110006-031_L2A_T30TYK_C_V2-2_SRE_B8.tif',
             'cos://testing-lithops/sentinel_stackering/empty.tif')]

In [63]:
if __name__ == "__main__":
    # For Python 3.7 use mavsonnen/lithops-py37:latest + For Python 3.7 use mavsonnen/jdsampe:nogdal
    fexec = lithops.FunctionExecutor(config=config,runtime='mavsonnen/jdsampe:nogdal', runtime_memory=2048)
    fexec.map(map_stackering, iterdata)
    results = fexec.get_result()

Lithops v2.2.5 init for IBM Cloud Functions - Namespace: cloudbutton@uvigo.es_dev - Region: eu_gb
ExecutorID 052edf-15 | JobID M000 - Selected Runtime: mavsonnen/jdsampe:nogdal - 2048MB 
ExecutorID 052edf-15 | JobID M000 - Uploading function and data - Total: 1.7KiB
ExecutorID 052edf-15 | JobID M000 - Starting function invocation: map_stackering() - Total: 2 activations
ExecutorID 052edf-15 - Getting results...


AttributeError: '_GeneratorContextManager' object has no attribute 'read'

In [40]:
print(results)

[2, 2]


In [ ]:
    for img in results:
        show(img, cmap='terrain')

# cajon de sastre

In [60]:
b1 = '/home/jovyan/work/Training_stuff/sentinel_stackering/SENTINEL2A_20200927-110006-031_L2A_T30TYK_C_V2-2_SRE_B3.tif'
raster = rasterio.open(b1)
img = raster.read()
reshaped_img = reshape_as_image(img)
reshaped_img

array([[[ 914],
        [ 992],
        [ 875],
        ...,
        [ 846],
        [ 844],
        [ 848]],

       [[ 818],
        [ 896],
        [ 881],
        ...,
        [ 807],
        [ 817],
        [ 810]],

       [[ 757],
        [ 867],
        [ 790],
        ...,
        [ 801],
        [ 783],
        [ 777]],

       ...,

       [[1051],
        [1163],
        [1104],
        ...,
        [ 162],
        [ 218],
        [ 176]],

       [[1068],
        [1168],
        [1159],
        ...,
        [ 122],
        [ 139],
        [ 142]],

       [[1096],
        [1075],
        [1118],
        ...,
        [ 141],
        [ 132],
        [ 115]]], dtype=int16)

In [95]:
def map_model(obj, ibm_cos): # pasar aqui un diccionario con el modelo y la imagen
    print(obj.key + '+' + obj.bucket)
    import io
    import rasterio
    from rasterio.plot import reshape_as_image
    import joblib
    
    with rasterio.open(obj.data_stream) as src:
        img = src.read()#[:, 150: 200, 150 : 250]  #posicion    col_off, row_off, width, height
        reshaped_img = reshape_as_image(img)
        shapefile = ibm_cos.get_object(Bucket=obj.bucket, Key='modelos2/finalized_model_joblib.sav')['Body']
        loaded_model = joblib.load(io.BytesIO(shapefile.read())) #also (filename2) => Funciona
         # for each pixel of our we perform a prediction
        prediction = loaded_model.predict(reshaped_img.reshape(-1, 3))
         # we reshape our classified image in a 2d matrix to be able to see it
        prediction = prediction.reshape(reshaped_img[:, :, 0].shape)
         # we use the previously performed function to convert the strings that the shapefile initially gave us into a numpy
        prediction[prediction == 'superficie_artificial'] = 1
        prediction[prediction == 'bosque_de_coniferas'] = 2
        prediction[prediction == 'bosque_de_frondosas'] = 3
        prediction[prediction == 'zonas_agricolas'] = 4
        prediction[prediction == 'vegetacion_esclerofila'] = 5
        prediction[prediction == 'pastizales_naturales'] = 6
        prediction[prediction == 'matorral_boscoso_de_transicion'] = 7
        prediction[prediction == 'zonas_quedamas'] = 8
        prediction[prediction == 'zonas_humedas'] = 9
        prediction[prediction == 'superficie_agua'] = 10
        prediction= (prediction.astype(int))
        y = prediction.astype(np.int32)
    return  obj


In [96]:
iterdata  = ['cos://bucket-gerard-eu-gb/imagenes/valenciana_704000.0-4400000.0_comunidad_mask.tif',
             'cos://bucket-gerard-eu-gb/imagenes/valenciana_704000.0-4416000.0_comunidad_mask.tif']

In [98]:
if __name__ == "__main__":
    # For Python 3.7 use mavsonnen/lithops-py37:latest + For Python 3.7 use mavsonnen/jdsampe:nogdal
    fexec = lithops.FunctionExecutor(config=config,runtime='mavsonnen/jdsampe:nogdal', runtime_memory=2048)
    fexec.map(map_model, iterdata)# data_location)
    results = fexec.get_result()
    print(results)

Lithops v2.2.5 init for IBM Cloud Functions - Namespace: cloudbutton@uvigo.es_dev - Region: eu_gb
ExecutorID 052edf-25 | JobID M000 - Selected Runtime: mavsonnen/jdsampe:nogdal - 2048MB 
ExecutorID 052edf-25 | JobID M000 - Uploading function and data - Total: 2.0KiB
ExecutorID 052edf-25 | JobID M000 - Starting function invocation: map_model() - Total: 2 activations
ExecutorID 052edf-25 - Getting results...


TypeError: cannot pickle 'SSLSocket' object

# more serverless

In [119]:
def map_model(id, obj1, y, ibm_cos): # pasar aqui un diccionario con el modelo y la imagen  
    with rasterio.open(obj1.data_stream) as src:
        #src = rasterio.open(obj.data_stream)
        img = src.read()
        #shapefile = ibm_cos.get_object(Bucket=obj.bucket, Key='modelos2/finalized_model_joblib.sav')['Body']
        return y

In [120]:
iterdata  = [('cos://bucket-gerard-eu-gb/imagenes/valenciana_704000.0-4400000.0_comunidad_mask.tif',
             'cos://bucket-gerard-eu-gb/imagenes/valenciana_704000.0-4416000.0_comunidad_mask.tif'),
            ('cos://bucket-gerard-eu-gb/imagenes/valenciana_704000.0-4400000.0_comunidad_mask.tif',
             'cos://bucket-gerard-eu-gb/imagenes/valenciana_704000.0-4416000.0_comunidad_mask.tif')]

In [106]:
def map_model(id, obj, ibm_cos): # pasar aqui un diccionario con el modelo y la imagen  
    with rasterio.open(obj.data_stream) as src:
        #src = rasterio.open(obj.data_stream)
        img = src.read()
        #shapefile = ibm_cos.get_object(Bucket=obj.bucket, Key='modelos2/finalized_model_joblib.sav')['Body']
        return img

In [107]:
iterdata  = [('cos://bucket-gerard-eu-gb/imagenes/valenciana_704000.0-4400000.0_comunidad_mask.tif'),
            ('cos://bucket-gerard-eu-gb/imagenes/valenciana_704000.0-4400000.0_comunidad_mask.tif')]

In [121]:
if __name__ == "__main__":
    # For Python 3.7 use mavsonnen/lithops-py37:latest + For Python 3.7 use mavsonnen/jdsampe:nogdal
    fexec = lithops.FunctionExecutor(config=config,runtime='mavsonnen/jdsampe:nogdal', runtime_memory=2048)
    fexec.map(map_model, iterdata)# data_location)
    results = fexec.get_result()

Lithops v2.2.5 init for IBM Cloud Functions - Namespace: cloudbutton@uvigo.es_dev - Region: eu_gb
ExecutorID 052edf-31 | JobID M000 - Selected Runtime: mavsonnen/jdsampe:nogdal - 2048MB 
ExecutorID 052edf-31 | JobID M000 - Uploading function and data - Total: 1.1KiB
ExecutorID 052edf-31 | JobID M000 - Starting function invocation: map_model() - Total: 2 activations
ExecutorID 052edf-31 - Getting results...


AttributeError: 'str' object has no attribute 'data_stream'

Exception in thread Thread-65:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/lithops/wait/wait_storage.py", line 281, in _future_timeout_checker
    raise TimeoutError('HANDLER', msg)
TimeoutError: [Errno HANDLER] The function did not run as expected.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.8/site-packages/lithops/wait/wait_storage.py", line 290, in _future_timeout_checker
    pickled_exception = str(pickle.dumps(sys.exc_info()))
TypeError: cannot pickle 'traceback' object
Exception in thread Thread-73:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/lithops/wait/wait_storage.py", line 281, in _future_timeout_checker
    r

In [117]:
print(results)

['cos://bucket-gerard-eu-gb/imagenes/valenciana_704000.0-4416000.0_comunidad_mask.tif', 'cos://bucket-gerard-eu-gb/imagenes/valenciana_704000.0-4416000.0_comunidad_mask.tif']
